In [2]:
import json
import os
from datetime import timedelta
import time
from sklearn.metrics import accuracy_score
import pandas as pd

# My modules
from types import SimpleNamespace
from common_utils import set_seeds
from config import Configuration
from train_manager import TrainManager

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [3]:
def run_inference(experiment_name, kaggle, config):
    experiment_dir = os.path.abspath(config.save_dir + f'/{experiment_name}')

    if not kaggle: 
        with open(experiment_dir + '/experiment_config.json', 'r') as f:
            config = json.load(f, object_hook=lambda d: SimpleNamespace(**d))
            set_seeds(config.seed)
        df = pd.read_csv(experiment_dir + '/holdout.csv', engine='python')
    else: 
        df = pd.DataFrame()
        df['image_id'] = list(os.listdir(config.test_img_dir))

    inference_start = time.time()
    
    # get predictions... folds = None means ensemble inference
    manager = TrainManager(folds_df=None, holdout_df=df, config=config,
                           experiment_dir=experiment_dir, experiment_name=experiment_name, kaggle=kaggle)
    manager.run_ensemble()
    
    print(f"Inference time: {str(timedelta(seconds=time.time() - inference_start))}")
    if not kaggle:
        acc = accuracy_score(y_true=df.label.values, y_pred=manager.test_predictions_ensembled)
        print("Ensemble holdout accuracy", acc)
    if kaggle:
         # make submission file
        submission = pd.DataFrame()
        submission['image_id'] = df['image_id']
        submission['label'] = manager.test_predictions_ensembled
        submission.to_csv('submission.csv', index=False)

In [4]:
run_inference(experiment_name='exp8_sgd_frozen_batch_norm', kaggle=False, config=Configuration())

FileNotFoundError: [Errno 2] No such file or directory: '/home/felix/dev/kaggle/kaggle-cassava/trained-models/exp8_sgd_frozen_batch_norm/experiment_config.json'